In [30]:
from itertools import accumulate
from bisect import *
# マージソートツリー (Merge Sort Tree)
# a: 配列
class MergeSortTree:
    def __init__(self, a, acc=False):
        self._n = len(a) # 配列の要素数
        self._log = (len(a)-1).bit_length() # 木の深さ(根は0)
        self._size = 1 << self._log # 葉の数
        # マージソートツリー(0番目の要素は使わない，1番目の要素が根)
        self._data = [None] * self._size + [[tmp] for tmp in a] + [[] for _ in range(self._size - self._n)]
        for i in range(self._size-1, 0, -1):
            self._data[i] = self._merge(self._data[i << 1], self._data[i << 1 | 1])
        if acc:
            self._acc = [None] + [list(accumulate([0]+tmp)) for tmp in self._data[1:]]
    def _merge(self, l, r):
        res = []
        i = j = 0
        while i < len(l) and j < len(r):
            if l[i] < r[j]:
                res.append(l[i])
                i += 1
            else:
                res.append(r[j])
                j += 1
        res += l[i:]
        res += r[j:]
        return res
    # [l, r) の範囲で k 未満の要素をソート済み配列で返す
    def less_than_sorted(self, l, r, k):
        sorted_l = []
        sorted_r = []
        l += self._size
        r += self._size
        while l < r:
            if l & 1:
                sorted_l = self._merge(sorted_l, [tmp for tmp in self._data[l] if tmp < k])
                l += 1
            if r & 1:
                r -= 1
                sorted_r = self._merge(sorted_r, [tmp for tmp in self._data[r] if tmp < k])
            l >>= 1
            r >>= 1
        return self._merge(sorted_l, sorted_r)
    # [l, r) の範囲で k 番目(0-indexed)に小さい要素を返す
    def nth_smallest(self, l, r, k):
        return self.less_than_sorted(l, r, float('inf'))[k]
    # [l, r) の範囲で k 未満の要素の個数を返す
    def count_less_than(self, l, r, k):
        l += self._size
        r += self._size
        cnt = 0
        while l < r:
            if l & 1:
                cnt += bisect_left(self._data[l], k)
                l += 1
            if r & 1:
                r -= 1
                cnt += bisect_left(self._data[r], k)
            l >>= 1
            r >>= 1
        return cnt
    # [l, r) の範囲で k 以上の要素の個数を返す
    def count_more_than(self, l, r, k):
        return r - l - self.count_less_than(l, r, k)
    # [l, r) の範囲で k 以上 m 未満の要素の個数を返す
    def count_between(self, l, r, k, m):
        l += self._size
        r += self._size
        cnt = 0
        while l < r:
            if l & 1:
                cnt += bisect_left(self._data[l], m) - bisect_left(self._data[l], k)
                l += 1
            if r & 1:
                r -= 1
                cnt += bisect_left(self._data[r], m) - bisect_left(self._data[r], k)
            l >>= 1
            r >>= 1
        return cnt
    # [l, r) の範囲で k 未満の要素の和を返す
    def sum_less_than(self, l, r, k):
        l += self._size
        r += self._size
        s = 0
        while l < r:
            if l & 1:
                s += self._acc[l][bisect_left(self._data[l], k)]
                l += 1
            if r & 1:
                r -= 1
                s += self._acc[r][bisect_left(self._data[r], k)]
            l >>= 1
            r >>= 1
        return s

def main():
    N = int(input())
    A = list(map(int, input().split()))
    mst = MergeSortTree(A, acc=True)
    Q = int(input())
    ans = 0
    for _ in range(Q):
        a,b,c = map(int, input().split())
        l,r,x = a^ans,b^ans,c^ans
        l -= 1
        x += 1
        ans = mst.sum_less_than(l, r, x)
        print(ans)

if __name__ == "__main__":
    main()

9
